# Project Name
This project notebook intends to predict, Given a dataset of clients, which is more likely to accept and invite and buy
the companies product.
We will demonstrate and explain the analysis and explain our method of work.

Install relevant packages.

In [55]:
%pip install scikit-learn
%pip install xgboost
%pip install tqdm

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 23.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 23.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 23.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Imports

In [56]:
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectPercentile
from sklearn.utils import class_weight
import xgboost as xgb
import pandas as pd
from tqdm import tqdm


### First Step
In order to use the 'reviews' data sets in our complete prediction model. We will need to predict the raiting of each review
the same way we did in the bonus assignment. The predicted rating will help us in taking advantage of the review data.
- The model will be built the same way as it was built in the bonus assignment.
- The data used will be the data given in the bonus assignment.
- The trained model will be used to predict the rating in the reviews_rollout.csv and reviews_training.csv
- The act of training the model on data X and predicting on data was is valid becuase the populations are the same.

In [57]:
def train_bonus_ass_model():
    """
        The function repeats the training process of the model in the
        Bonus assignment. Returned values are the trained model and the selected values.

        Features selection is done by '''SelectPercentile'''.
        Model used is XGBoost.
        Test is 20% of total data.
    """
    df = pd.read_csv("../BonusAssignment/text_training.csv", usecols=list(range(1,2002)))

    # separate the features and target variable
    features = df.iloc[:, 1:-1] # all columns except the last one (rating)
    labels = df.iloc[:, -1] # last column (rating)

    selector = SelectPercentile(percentile=10)
    x = selector.fit_transform(features, labels)
    selected_features = selector.get_feature_names_out()

    x_train, x_test, y_train, y_test = train_test_split(x, labels, test_size=0.2, random_state=1)

    xgb_bonus_clf = xgb.XGBClassifier(max_depth=4, seed=2)
    xgb_bonus_clf.fit(x_train, y_train)

    return xgb_bonus_clf, selected_features

In [58]:
def bonus_ass_rating_prediction(table_path, trained_model, select_features):
    """
        The function Receives the path to one of the foloowing tables: 'reviews_rollout.csv'
        or 'reviews_training.csv' and returns the predicted values in dataframe along with the
        respected ids.

        Assumptions:
        1. Both tables contain an id column
        2. Rating Column does not appear in any table
        3. model was Trained on the same tryp of population
    """    
    df = pd.read_csv(table_path)
    selected = df[select_features]
    y_pred = trained_model.predict(selected)

    return pd.concat([df["ID"], pd.Series(y_pred, name='Rating')], axis=1)

#### Train and Predict - Bonus model
The following section uses the functions above to predict the raiting column in the given reviews tables

In [59]:
trained_model, selected_features = train_bonus_ass_model()
y_pred_rev_training =   bonus_ass_rating_prediction("Documents/reviews_training.csv", trained_model, selected_features)
y_pred_rev_rollout =    bonus_ass_rating_prediction("Documents/reviews_rollout.csv", trained_model, selected_features)


c:\Users\Coffe\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: UserWarning: Features [1208 1527] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
c:\Users\Coffe\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


#### Merge tables
- ffp_rollout_X
- ffp_train

The merge process has one critical issue: Not all ID's in the ffp table, exist in the reviews table, That is when when performing an ```outer join```, the combined data will have ```Nan``` values in the ```Rating``` column.
We will deal with this issue in the next step.

In [60]:
df_ffp_train = pd.read_csv("Documents/ffp_train.csv")
merged_ffp_train = pd.merge(left=df_ffp_train, right=y_pred_rev_training,how="outer", on="ID")

df_ffp_rollout = pd.read_csv("Documents/ffp_rollout_X.csv")
merged_ffp_rollout = pd.merge(left=df_ffp_rollout, right=y_pred_rev_rollout,how="outer", on="ID")

## Step Two
### Choosing a model

In order to predict the ```Buyer flag``` of the ffp table, we will use the merged tables.
The problem: Some rows contain the rating value and some dont. To solve this, we will build model that is made out of two different models. One will work on the fpp table and one will work on the merged table. Predicted value will be chosen from one of the two results

### Model 1 - Table containing both rating and ffp data
Clear all ```Nan``` values from the table

In [61]:
# Devide the data according to the rating values.
# model1 will use df_ffp_data (data with no rating prediction)
# model2 will use df_combined_data (data with rating prediction)

#df_model1_train_no_rating = merged_ffp_train[merged_ffp_train["Rating"].isna()].drop("Rating", axis=1)
df_model2_train_both = merged_ffp_train[~merged_ffp_train["Rating"].isna()]


### Testing different models
We will try two algorithms and two feature selection methods:

In [62]:
# Strip ID and label
features = df_model2_train_both.drop(["BUYER_FLAG","ID"], axis=1)
# Extract the label column
label = df_model2_train_both["BUYER_FLAG"]


##### Feature selection methods
- One uses a correlation matrix to pick the most corralated features with the label.
- The second uses the function ```SelectPercentile```

In [63]:
def select_features_by_corr(data, treshold):
    mat = data.corr().drop(["BUYER_FLAG","ID"], axis=0)
    mat = mat[abs(mat["BUYER_FLAG"]) > treshold]
    return list(mat.index)

In [64]:
from sklearn.feature_selection import SelectPercentile

def select_features_by_SelecPer(features, lable, p):
    selector = SelectPercentile(percentile=p)
    selector.fit_transform(features, label)
    return selector.get_feature_names_out()

Studying the label properties

In [65]:
data = df_model2_train_both.iloc[:,1:] # Get rid of ID col

mat = data.corr().drop("BUYER_FLAG", axis=0)
mat["BUYER_FLAG"] = abs(mat["BUYER_FLAG"])
print(f"Minimum corrlation value with 'BUYER_FLAG': {mat.describe()['BUYER_FLAG'].loc['min']}")
print(f"Maximum corrlation value with 'BUYER_FLAG': {mat.describe()['BUYER_FLAG'].loc['max']}")

Minimum corrlation value with 'BUYER_FLAG': 0.0031826178888089756
Maximum corrlation value with 'BUYER_FLAG': 0.560140516762669


##### Implement function to wrap the full training and model selection

In [66]:
def get_best_xgboost(features, label):
    
    parameters = {'score': 0,
                  'features': [],
                  'model': [],
                  'depth': -1
                  }        
    
    x_train, x_test, y_train, y_test = train_test_split(features, label, test_size=0.2, random_state=1)

    for d in range(1, 10, 1):
        classes_weights = class_weight.compute_sample_weight(
            class_weight='balanced',
            y=y_train
        )

        xgbclf = xgb.XGBClassifier(max_depth=d, seed=2)
        xgbclf.fit(x_train, y_train, sample_weight=classes_weights)
        y_pred = xgbclf.predict(x_test)
        score = f1_score(y_test, y_pred)
                
        if score > parameters['score']:
            parameters['score'] = score
            parameters['model'] = xgbclf
            parameters['depth'] = d
            #st = f"The best XGBoost model...\nHas a treshold of {treshold}\nUses {len(selected_features)} features\nHas a model depth of {d}\nand an F1 score of {best_score}"
            #st += f"\nFeatures selected: {best_features}"
            
    return parameters

In [67]:
from sklearn.ensemble import RandomForestClassifier

def get_best_rf(features, label):

    parameters = {'score': 0,
                  'features': [],
                  'model': [],
                  'n_estimators': -1
                } 
    
    x_train, x_test, y_train, y_test = train_test_split(features, label, test_size=0.2, random_state=1)
    
    for n_e in range(1, 200, 1):

        rf = RandomForestClassifier(n_estimators=n_e, class_weight='balanced')
        rf.fit(x_train, y_train)
        y_pred = rf.predict(x_test)
        score = f1_score(y_test, y_pred)

        if score > parameters['score']:
            parameters['score'] = score
            parameters['model'] = rf
            parameters['n_estimators'] = n_e
            
    return parameters    

lets run the different models.. (names are not correct)

In [68]:
def show_output(params):
    st = f"{50*'~'}\n"
    st += f"The Model is {type(params['model'])}.\n"
    st += f"The number of features picked is {len(params['features'])}.\n"
    
    if "depth" in params.keys():
        st += f"The depth selected is {params['depth']}\n"

    if "n_estimator" in params.keys():
        st += f"The number of estimators picked is {params['n_estimators']}\n."

    if "percentile" in params.keys():
        st += f"The percentile value is {params['percentile']}.\n"

    if "treshold" in params.keys():
        st += f"The treshold chosen is {params['treshold']}.\n"
    
    st += f"The F1 score is {params['score']}\n."
    st += f"{50*'~'}"
    
    print(st)

Trying XGBoost with Selection by corr

In [69]:
df_model2_train_both

,ID,CUSTOMER_GRADE,STATUS_PANTINUM,STATUS_GOLD,STATUS_SILVER,NUM_DEAL,LAST_DEAL,ADVANCE_PURCHASE,FARE_L_Y1,FARE_L_Y2,...,POINTS_L_Y2,POINTS_L_Y3,POINTS_L_Y4,POINTS_L_Y5,COUPON_FLAG,CANCEL_FLAG,CREDIT_FLAG,RELATED_FLAG,BUYER_FLAG,Rating
1,2,9.493459,0,0,0,5,26.7,27,120.7,112.0,...,112.5,104.6,110.9,137.3,0,0,1,0,0,0.0
13,14,12.200276,0,0,0,2,58.7,21,124.0,120.7,...,147.9,123.4,120.4,119.5,0,0,0,0,1,1.0
17,18,8.248704,0,0,1,6,24.0,28,143.3,145.3,...,167.3,163.3,169.4,152.3,0,0,0,0,0,0.0
92,93,13.308937,0,0,0,5,21.3,9,104.7,97.3,...,111.5,94.5,108.1,125.2,0,0,0,0,0,1.0
94,95,10.420929,0,0,0,6,16.0,15,90.0,90.7,...,90.3,117.7,109.7,97.2,0,0,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29937,29938,8.813226,0,0,0,7,13.3,22,87.3,80.7,...,96.4,78.8,88.8,100.9,0,1,0,0,0,0.0
29947,29948,12.060344,0,0,0,5,16.0,14,95.3,100.7,...,106.4,87.6,139.9,94.5,0,0,0,0,0,0.0
29968,29969,9.308334,0,1,0,5,48.0,22,218.7,224.0,...,192.6,219.3,218.7,223.8,0,0,0,1,1,1.0
29979,29980,12.411760,0,0,1,3,45.3,14,146.0,141.3,...,188.8,144.1,155.6,154.2,0,0,0,0,1,1.0


In [70]:
prev = -1
params = {}
best_params = {"score": 0}

for i in tqdm(range(1, 500, 1)):
    treshold = i / 1000
    selected = select_features_by_corr(df_model2_train_both, treshold)

    if prev == len(selected):
        continue
    prev = len(selected)

    params = get_best_xgboost(features[selected], label)
    if params['score'] > best_params['score']:
        best_params = params
        best_params['treshold'] = treshold
        best_params['features'] = selected
        
params_xgb_corr = best_params
show_output(params_xgb_corr)
params_xgb_corr

100%|██████████| 499/499 [00:29<00:00, 16.64it/s] 

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
The Model is <class 'xgboost.sklearn.XGBClassifier'>.
The number of features picked is 10.
The depth selected is 2
The treshold chosen is 0.027.
The F1 score is 0.6382978723404256
.~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


{'score': 0.6382978723404256,
 'features': ['STATUS_PANTINUM',
  'STATUS_GOLD',
  'STATUS_SILVER',
  'FARE_L_Y1',
  'FARE_L_Y4',
  'FARE_L_Y5',
  'POINTS_L_Y4',
  'POINTS_L_Y5',
  'COUPON_FLAG',
  'Rating'],
 'model': XGBClassifier(base_score=None, booster=None, callbacks=None,
               colsample_bylevel=None, colsample_bynode=None,
               colsample_bytree=None, early_stopping_rounds=None,
               enable_categorical=False, eval_metric=None, feature_types=None,
               gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
               interaction_constraints=None, learning_rate=None, max_bin=None,
               max_cat_threshold=None, max_cat_to_onehot=None,
               max_delta_step=None, max_depth=2, max_leaves=None,
               min_child_weight=None, missing=nan, monotone_constraints=None,
               n_estimators=100, n_jobs=None, num_parallel_tree=None,
               predictor=None, random_state=None, ...),
 'depth': 2,
 'treshol

Using Random Forest with selection by corr

In [71]:
prev = -1
params = {}
best_params = {"score": 0}

for p in tqdm(range(1, 101, 1)):
    selected = select_features_by_SelecPer(features, label, p)

    if prev == len(selected):
        continue
    prev = len(selected)

    params = get_best_xgboost(features[selected], label)
    if params['score'] > best_params['score']:
        best_params = params
        best_params['percentile'] = p
        best_params['features'] = selected

params_xgb_per = best_params
show_output(params_xgb_per)

100%|██████████| 100/100 [00:26<00:00,  3.73it/s]

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
The Model is <class 'xgboost.sklearn.XGBClassifier'>.
The number of features picked is 10.
The depth selected is 2
The percentile value is 43.
The F1 score is 0.6382978723404256
.~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


We will now repeat the algorithmes above, using the select percentile function.

In [72]:
prev = -1
params = {}
best_params = {"score": 0}

for i in tqdm(range(1, 500, 1)):
    treshold = i / 1000
    selected = select_features_by_corr(df_model2_train_both, treshold)

    if prev == len(selected):
        continue
    prev = len(selected)

    params = get_best_rf(features[selected], label)
    if params['score'] > best_params['score']:
        best_params = params
        best_params['treshold'] = treshold
        best_params['features'] = selected

params_rf_corr = best_params
show_output(params_rf_corr)

100%|██████████| 499/499 [41:09<00:00,  4.95s/it]  

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
The Model is <class 'sklearn.ensemble._forest.RandomForestClassifier'>.
The number of features picked is 5.
The treshold chosen is 0.031.
The F1 score is 0.6326530612244898
.~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


Now the RandomForest with the selectPercentile

In [73]:
prev = -1
params = {}
best_params = {"score": 0}

for p in tqdm(range(1, 101, 1)):
    selected = select_features_by_SelecPer(features, label, p)

    if prev == len(selected):
        continue
    prev = len(selected)

    params = get_best_rf(features[selected], label)
    if params['score'] > best_params['score']:
        best_params = params
        best_params['percentile'] = p
        best_params['features'] = selected

params_rf_per = best_params
show_output(params_rf_per)

 81%|████████  | 81/100 [37:15<10:21, 32.73s/it] 

Picking the best model:

In [ ]:
model1 = ""

### Model 2 - Table containing ffp data only

Need to rerun models....

In [ ]:
df_ffp_train

In [ ]:
features = df_ffp_train.drop(["ID", "BUYER_FLAG"], axis=1)
label = df_ffp_train["BUYER_FLAG"]

In [ ]:
prev = -1
params = {}
best_params = {"score": 0}

for i in tqdm(range(1, 500, 1)):
    treshold = i / 1000
    selected = select_features_by_corr(df_ffp_train,treshold)

    if prev == len(selected):
        continue
    prev = len(selected)

    params = get_best_xgboost(features[selected], label)
    if params['score'] > best_params['score']:
        best_params = params
        best_params['treshold'] = treshold
        best_params['features'] = selected

params_xgb_corr = best_params
show_output(params_xgb_corr)

In [ ]:
prev = -1
params = {}
best_params = {"score": 0}

for p in tqdm(range(1, 101, 1)):
    selected = select_features_by_SelecPer(features, label, p)

    if prev == len(selected):
        continue
    prev = len(selected)

    params = get_best_xgboost(features[selected], label)
    if params['score'] > best_params['score']:
        best_params = params
        best_params['percentile'] = p
        best_params['features'] = selected

params_xgb_per = best_params
show_output(params_xgb_per)

In [ ]:
prev = -1
params = {}
best_params = {"score": 0}

for i in tqdm(range(1, 500, 1)):
    treshold = i / 1000
    selected = select_features_by_corr(df_ffp_train,treshold)

    if prev == len(selected):
        continue
    prev = len(selected)

    params = get_best_rf(features[selected], label)
    if params['score'] > best_params['score']:
        best_params = params
        best_params['treshold'] = treshold
        best_params['features'] = selected

params_rf_corr = best_params
show_output(params_rf_corr)

In [ ]:
prev = -1
params = {}
best_params = {"score": 0}

for p in tqdm(range(1, 101, 1)):
    selected = select_features_by_SelecPer(features, label, p)

    if prev == len(selected):
        continue
    prev = len(selected)

    params = get_best_rf(features[selected], label)
    if params['score'] > best_params['score']:
        best_params = params
        best_params['percentile'] = p
        best_params['features'] = selected

params_rf_per = best_params
show_output(params_rf_per)

### Choosing the "Winning" models


In [ ]:
model1 = ''
model2 = ''

recommandations = pd.DataFrame(columns="BUYER_FLAG")

def combined_model(row):
    x = row.drop(["BUYER_FLAG"], axis=1)
    if x["Rating"].isna():
        return model2.predict(x.drop(["Rating"], axis=1))
    return model1.predict(x)

recommandations["BUYER_FLAG"] = pd.apply(data=merged_ffp_rollout, combined_model)